<h1><center>Training Custom Classifiers with IBM Watson Visual Recognition in Python</center></h1>

<center>This notebook is about how to operate the Watson Visual Recognition API using the Python Programming Language. The advantage of using the Watson Visual Recognition API over the Graphic User Interface on the Browser that you did earlier in this course is because you can automate the training, and testing of your Visual Recognition model.</center>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ol>
    <li><a href="#ref3">Training the Classifier</a></li>
    <li><a href="#ref4">Testing the Classifier</a></li>
</ol>    
</div>

<p>In this lab, we will be creating a completely new image classifier using training images. We will train a custom classifier to identify between three different dog breeds (Golden Retriever, Beagle and Husky).</p>

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/dog-breed.png" width="480"/>

In [1]:
# Paste your API key for IBM Watson Visual Recognition below:
my_apikey = 'aW7nYwlWoElVvaQGwaAWQs6mDg-XRbnHYxzCXaE-ehTD'

<h4>Initialize Watson Visual Recognition</h4>
Lets create your own Watson Visual Recognition instance, it will allow you to make calls to the Watson Visual Recognition API.

In [2]:
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator(my_apikey)

visrec = VisualRecognitionV3('2018-03-19', 
                             authenticator=authenticator)


<p>We are going to train an Image Recognition model to classify different types of dog. The dataset that we are going to use are the zip files that we use below</p>

<ul>
    <li>beagle.zip</li>
    <li>husky.zip</li>
    <li>golden-retriever.zip</li>
</ul>

<a id="ref3"></a>
<h2>Training Classifier</h2>

<h4>Download the differerent breed of dog images as zip files</h4>
<p>We will use the <b>urlretrieve</b> method from the <b>urllib.request</b> library to download the dataset above.</p> 

In [3]:
import urllib.request

# Downloading Beagle dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Beagle.zip", 
                           "beagle.zip")

# Downloading Husky dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/Husky.zip", 
                           "husky.zip")

# Downloading Golden Retriever dataset
urllib.request.urlretrieve("http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/GoldenRetriever.zip", 
                           "goldenretriever.zip") #note that we should remove any hyphens from the zip file name

('goldenretriever.zip', <http.client.HTTPMessage at 0x1994ec32248>)

<p>Lets train our Visual Recognition model to recognize the three breeds of dogs using the <b>create_classifier</b> method from the Watson Image Recognition API.</p>

In [3]:
import json
with open('beagle.zip', 'rb') as beagle, \
     open('goldenretriever.zip', 'rb') as gretriever, \
     open('husky.zip', 'rb') as husky:
        response = visrec.create_classifier(name="dogbreedclassifier",
                                        positive_examples={'beagle': beagle, \
                                                           'goldenretriever': gretriever, \
                                                           'husky': husky})
print(json.dumps(response.get_result(), indent=2))

{
  "classifier_id": "dogbreedclassifier_634631513",
  "name": "dogbreedclassifier",
  "status": "training",
  "owner": "8bf32f0a-0e94-400e-bec0-113b4c81cc32",
  "created": "2020-04-30T09:46:49.468Z",
  "updated": "2020-04-30T09:46:49.468Z",
  "classes": [
    {
      "class": "husky"
    },
    {
      "class": "goldenretriever"
    },
    {
      "class": "beagle"
    }
  ],
  "core_ml_enabled": true
}


In [4]:
#lets grab the classifier id
classifier_id = response.get_result()["classifier_id"]
classifier_id

'dogbreedclassifier_634631513'

<div style="background-color: #fcf2f2">
    <h2>Note!</h2> 
    <p>If you receive the following error.</p>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/Error-Code.png" width="1080">
    <p>It means that you have more than 1 Visual Recognition Instances running on your lite plan, and the lite plan only allows for no more than 2 Visual Recognition instances. So you might want to delete one of your custom classifier in your Watson Visual Recognition Instance.</p>
        <p>Log into your IBM Cloud Account with the following link.</p> <p><a href="https://cocl.us/CV0101EN_IBM_Cloud_Login">https://cloud.ibm.com</a></p>
    <ol>
        <li>Click on <b>Services</b></li>
        <li>Under Services, click on your Watson Visual Recognition Instance</li>
        <li>Then click on Create a Custom Model</li>
        <li>Then delete one of your Custom Visual Recognition Model</li>
    </ol>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/Delete-Instance.png" width="680">
</div>
 

<h4>Is the model still training?</h4>
Depending on the number of images, it may take <b>a Several Minutes </b> for Watson to build a custom classifier. Please wait tell you get <b> Good to Go<b> 

In [7]:
Status = visrec.get_classifier(classifier_id=classifier_id, verbose=True).get_result()['status']
if Status=='training': 
    print ("Please, Wait to complete training.......")
else:
    print("Good to go ")

Good to go 


<h4>List all (custom) classifiers</h4>

<h4>If the status is still training, please rerun the above cell and wait until you see ready</h4> 

In [8]:
visrec.list_classifiers(verbose=True).get_result()

{'classifiers': [{'classifier_id': 'dogbreedclassifier_634631513',
   'name': 'dogbreedclassifier',
   'status': 'ready',
   'owner': '8bf32f0a-0e94-400e-bec0-113b4c81cc32',
   'created': '2020-04-30T09:46:49.468Z',
   'updated': '2020-04-30T09:46:49.468Z',
   'classes': [{'class': 'husky'},
    {'class': 'goldenretriever'},
    {'class': 'beagle'}],
   'core_ml_enabled': True}]}

<a id="ref4"></a>
<h2>Testing Classifier</h2>
<p>Let's test the classifier, the function <b>getdf_visrec</b> below uses the method <b>classify</b> from Watson Visual Recognition API to upload the image to the classifier and give us a result in JSON(JavaScript Object Notation) format. Then we use the method <b>json_normalize</b> from the "Pandas" library in Python to turn the result into a table because it is more human readable.</p>

In [9]:
from pandas.io.json import json_normalize

def getdf_visrec(url, classifier_ids, apikey = my_apikey):
    
    json_result = visrec.classify(url=url,
                              threshold='0.6',
                              classifier_ids=classifier_id).get_result()
    
    json_classes = json_result['images'][0]['classifiers'][0]['classes']
    
    df = json_normalize(json_classes).sort_values('score', ascending=False).reset_index(drop=True)
    
    return df

<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/GoldenRetriever1_stacked.jpg">
<p>Let's test our Visual Recognition model on this picture of Golden Retriever</p>

<p>Please wait for your Custom Model to finish training before you upload your test image to your Custom Classifier, <b>you might get an error if your model is still training and you run the function below.</p>
<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Images/Training.png" width="680">

In [10]:
getdf_visrec(url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/GoldenRetriever1_stacked.jpg',
            classifier_ids=classifier_id)

,class,score
0,goldenretriever,0.908


<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/cat-2083492_960_720.jpg">
<p>Lets test our Visual Recognition model on this picture of cat</p>

In [11]:
getdf_visrec(url = 'http://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/CV0101/Dataset/cat-2083492_960_720.jpg',
            classifier_ids=classifier_id)

,class,score
0,husky,0.821


<p>Our model will mis-classify the cat in the picture because our custom visual recognition model is only trained for recognizing different breeds of dogs.</p>

<h4>Delete all classifiers</h4>
<p>If you want to delete you classifiers, lets get the classifier id tht you want to delete. The method <b>list_classifiers</b> from Watson Visual Recognition API list all the classifier in your IBM Cloud account.</p>

In [12]:
import json

classifiers = visrec.list_classifiers(verbose=True).get_result()['classifiers']
print(json.dumps(classifiers, indent=2))

[
  {
    "classifier_id": "dogbreedclassifier_634631513",
    "name": "dogbreedclassifier",
    "status": "ready",
    "owner": "8bf32f0a-0e94-400e-bec0-113b4c81cc32",
    "created": "2020-04-30T09:46:49.468Z",
    "updated": "2020-04-30T09:46:49.468Z",
    "classes": [
      {
        "class": "husky"
      },
      {
        "class": "goldenretriever"
      },
      {
        "class": "beagle"
      }
    ],
    "core_ml_enabled": true
  }
]


Just paste your classifier id and it will be deleted

In [13]:
mycid = 'dogbreedclassifier_634631513' #the classifier id you want to delete
visrec.delete_classifier(classifier_id = mycid)

<p>After training your custom classifier, you might want to delete all the custom classifiers from your account, write a piece of code to use the *list_classifier* and *delete_classifier* method to delete all the custom classifiers in your account.</p>

In [14]:
for i in visrec.list_classifiers().get_result()['classifiers']:
    print('Deleting ...' + i['classifier_id'])
    visrec.delete_classifier(classifier_id=i['classifier_id'])